In [1]:
# import gensim
import os
import collections
# import smart_open
import random
import pandas as pd
import numpy as np

In [2]:
os.getcwd()

'/home/ppas/Documents'

In [3]:
# xls = pd.ExcelFile('test_and_golden_reference_data.xlsx')

In [4]:
golden_csv = pd.read_csv('golden_data.csv', header=None)


In [5]:
sample_csv = pd.read_csv('test_data.csv')


In [6]:
golden_csv.columns = [['name', 'mcn-id','existing-or-new','address1','address2','city','zip','country','temp1','temp2','temp3','temp4','temp5','website','temp6','temp7']]
# golden_csv.dropna(axis=1, how='any')
golden_csv.head()

,name,mcn-id,existing-or-new,address1,address2,city,zip,country,temp1,temp2,temp3,temp4,temp5,website,temp6,temp7
0,Greg Pollom,MCN-47996044,EC-Existing,218 Fulford Ave,Bel Air,MD,21014,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Little Apple Toyota Honda,MCN-05952813,R-VAR,2828 Amherst Ave,Manhattan,KS,66502,US,2828 Amherst Ave,Manhattan,KS,66502,US,NaN,NaN,NaN
2,Adp Network Solutions 76002379,MCN-67486028,EC-Existing,16177 Sw 155 Ave,Miami,FL,33187,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Prestige Subaru Suzuki 08320005,MCN-09000630,EC-Existing,585 Tunnel Rd,Asheville,NC,28805,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wolberg Electrical Supply Company I,MCN-30243660,EC-Existing,35 Industrial Park Road,Albany,NY,12206,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
golden_csv_cleaned = golden_csv.drop('temp1', axis=1)
golden_csv_cleaned = golden_csv_cleaned.drop('temp2', axis=1)
golden_csv_cleaned = golden_csv_cleaned.drop('temp3', axis=1)
golden_csv_cleaned = golden_csv_cleaned.drop('temp4', axis=1)
golden_csv_cleaned = golden_csv_cleaned.drop('temp5', axis=1)
golden_csv_cleaned = golden_csv_cleaned.drop('temp6', axis=1)
golden_csv_cleaned = golden_csv_cleaned.drop('temp7', axis=1)
# golden_csv_cleaned = golden_csv_cleaned.dropna(axis=0, how='any')
values = {'existing-or-new': 'EC-Existing', 'city': 'TX', 'zip': 11111, 'website': 'www.google.com'}
golden_csv_cleaned['existing-or-new'] = golden_csv_cleaned['existing-or-new'].fillna('EC-Existing')
golden_csv_cleaned['city'] = golden_csv_cleaned['city'].fillna('TX')
golden_csv_cleaned['zip'] = golden_csv_cleaned['zip'].fillna('11111')
golden_csv_cleaned['website'] = golden_csv_cleaned['website'].fillna('www.google.com')
golden_csv_cleaned.isnull().sum()

/home/ppas/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


name               0
mcn-id             0
existing-or-new    0
address1           0
address2           0
city               0
zip                0
country            0
website            0
dtype: int64

In [8]:
len(golden_csv_cleaned)

1033

In [9]:
golden_csv_cleaned.head()

,name,mcn-id,existing-or-new,address1,address2,city,zip,country,website
0,Greg Pollom,MCN-47996044,EC-Existing,218 Fulford Ave,Bel Air,MD,21014,US,www.google.com
1,Little Apple Toyota Honda,MCN-05952813,R-VAR,2828 Amherst Ave,Manhattan,KS,66502,US,www.google.com
2,Adp Network Solutions 76002379,MCN-67486028,EC-Existing,16177 Sw 155 Ave,Miami,FL,33187,US,www.google.com
3,Prestige Subaru Suzuki 08320005,MCN-09000630,EC-Existing,585 Tunnel Rd,Asheville,NC,28805,US,www.google.com
4,Wolberg Electrical Supply Company I,MCN-30243660,EC-Existing,35 Industrial Park Road,Albany,NY,12206,US,www.google.com


In [10]:
sample_csv_cleaned = sample_csv.drop('eu_address2', axis=1)
sample_csv_cleaned = sample_csv_cleaned.drop('eu_address3', axis=1)
sample_csv_cleaned = sample_csv_cleaned.drop('eu_address4', axis=1)
sample_csv_cleaned = sample_csv_cleaned.drop('eu_province', axis=1)
sample_csv_cleaned = sample_csv_cleaned.dropna(axis=0, how='any')
sample_csv_cleaned.isnull().sum()

Shard ID       0
eu_mcn         0
eu_name        0
eu_address1    0
eu_city        0
eu_zip_code    0
eu_state       0
eu_country     0
dtype: int64

In [11]:
sample_csv_cleaned.head()

,Shard ID,eu_mcn,eu_name,eu_address1,eu_city,eu_zip_code,eu_state,eu_country
0,n1,MCN-47996044,Barwick Cpa,218 Fulford Ave,Bel Air,210140000,MD,US
1,n1,MCN-05952813,Little Apple Toyota Honda,2828 Amherst Ave,Manhattan,665023096,KS,US
2,n1,MCN-67486028,Adp Network Solutions 76002379,16177 Sw 155 Ave,Miami,331870000,FL,US
3,n1,MCN-09000630,Prestige Subaru Suzuki08320005,585 Tunnel Rd,Asheville,288050000,NC,US
4,n1,MCN-30243660,WOLBERG ELECTRICAL SUPPLY COMPANY I,35 INDUSTRIAL PARK ROAD,ALBANY,12206,NY,US


In [12]:
len(sample_csv_cleaned)

117

In [13]:
sample_csv_cleaned['name_address'] = sample_csv_cleaned['eu_name']+ ' ' + sample_csv_cleaned['eu_address1']+' '  + sample_csv_cleaned['eu_city']+ ' ' + \
sample_csv_cleaned['eu_zip_code']+ ' '  + sample_csv_cleaned['eu_state']+ ' ' + sample_csv_cleaned['eu_country']

In [14]:
a1 = golden_csv_cleaned['name'].values
a2 = golden_csv_cleaned['address1'].values
a3 = golden_csv_cleaned['address2'].values
a4 = golden_csv_cleaned['city'].values
a5 = golden_csv_cleaned['zip'].values
a6 = golden_csv_cleaned['country'].values

a = a1+' '+a2+' '+a3+' '+a4+' '+a5+' '+a6
# a = np.hstack((a1,a2,a3))

In [15]:
a

array([['Greg Pollom 218 Fulford Ave Bel Air MD 21014 US'],
       ['Little Apple Toyota Honda 2828 Amherst Ave Manhattan KS 66502 US'],
       ['Adp Network Solutions 76002379 16177 Sw 155 Ave Miami FL 33187 US'],
       ...,
       ['Linxens Holdings 6 Rue Helene Boucher Guyancourt . 78280 FR'],
       ['Dr. Steven Christie  M.d. 524 Hampton Ln Key Biscayne FL 33149 US'],
       ['Esbelt Agerhatten Odense SO SYDDANMARK 5220 DK']], dtype=object)

In [16]:
sample_csv_cleaned.head()

,Shard ID,eu_mcn,eu_name,eu_address1,eu_city,eu_zip_code,eu_state,eu_country,name_address
0,n1,MCN-47996044,Barwick Cpa,218 Fulford Ave,Bel Air,210140000,MD,US,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...
1,n1,MCN-05952813,Little Apple Toyota Honda,2828 Amherst Ave,Manhattan,665023096,KS,US,Little Apple Toyota Honda 2828 Amherst Ave Man...
2,n1,MCN-67486028,Adp Network Solutions 76002379,16177 Sw 155 Ave,Miami,331870000,FL,US,Adp Network Solutions 76002379 16177 Sw 155 Av...
3,n1,MCN-09000630,Prestige Subaru Suzuki08320005,585 Tunnel Rd,Asheville,288050000,NC,US,Prestige Subaru Suzuki08320005 585 Tunnel Rd A...
4,n1,MCN-30243660,WOLBERG ELECTRICAL SUPPLY COMPANY I,35 INDUSTRIAL PARK ROAD,ALBANY,12206,NY,US,WOLBERG ELECTRICAL SUPPLY COMPANY I 35 INDUSTR...


In [17]:
train_arr = []
test_arr = []
mcn_arr_test = []
train_data_length=len(sample_csv_cleaned)
for i in range(0, int(train_data_length)):
    train_arr_temp = sample_csv_cleaned['name_address'].values[i]
    mcn_number_temp = sample_csv_cleaned['eu_mcn'].values[i]
    train_arr.append(train_arr_temp)
    mcn_arr_test.append(mcn_number_temp)

            

In [70]:
# mcn_arr_test

In [18]:
mcn_arr_golden = golden_csv_cleaned['mcn-id'].values.tolist()

In [19]:
train_arr = np.array(train_arr)

# Uncomment later

In [20]:
# train_arr

In [21]:
np.savetxt(r'test_data_small.txt', train_arr, fmt='%s', encoding='utf-8')

In [22]:
np.savetxt(r'golden_data_small.txt', a, fmt='%s', encoding='utf-8')

# fuzzy wuzzy

In [23]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [24]:
golden_data = a.tolist()

In [25]:
golden_flat_list = [item for sublist in golden_data for item in sublist]

In [26]:
test_data = train_arr.tolist()

In [27]:
mcn_arr_test[3]

'MCN-09000630'

In [28]:
test_dict = {}
# golden_dict.setdefault('dummy',[])
for i in range(0, len(mcn_arr_test)):
    test_dict[mcn_arr_test[i]] = train_arr[i]
    


# Uncomment later

In [82]:
# test_dict

In [29]:
golden_dict = {}
# golden_dict.setdefault('dummy',[])
for i in range(0, len(mcn_arr_golden)):
    golden_dict[mcn_arr_golden[i][0]] = golden_flat_list[i]
    


# Uncomment later

In [84]:
# golden_dict

# Uncomment later the printf

In [31]:
import time
choices = golden_dict.values()
score= 0
found_flag=0
not_found=[]
t = time.time()
for i in range(0, len(test_data)-1):
    found_flag=0
    chlist=process.extract(test_data[i], choices, limit=2)
    print("{},   --test--{}".format(test_data[i], mcn_arr_test[i]))
    
    for mcn,value in golden_dict.items():
        for j in range(0,2):
            if value == chlist[j][0]:
                if mcn_arr_test[i] == mcn:
                    score+=1
                    
                    print("{},   --MDM--{}".format(chlist[j][0], mcn))
                    print("===================================================")
                    found_flag=1
                    break
    if found_flag == 0:
        not_found.append(i)
t1 = t-time.time()
print('SELFTIMED:', t1)        
print("Total matches found are {} out of {}".format(score, len(test_data)))        
print('accuracy is', 100* score/len(test_data), '%')
print('mismatched sample indexes in test data are:')
print(not_found)
print("===================================================")
print('We are done with 1st level of matching')

Barwick Cpa 218 Fulford Ave Bel Air 210140000 MD US,   --test--MCN-47996044
Little Apple Toyota Honda 2828 Amherst Ave Manhattan 665023096 KS US,   --test--MCN-05952813
Little Apple Toyota Honda 2828 Amherst Ave Manhattan KS 66502 US,   --MDM--MCN-05952813
Adp Network Solutions 76002379 16177 Sw 155 Ave Miami 331870000 FL US,   --test--MCN-67486028
Adp Network Solutions 76002379 16177 Sw 155 Ave Miami FL 33187 US,   --MDM--MCN-67486028
Prestige Subaru Suzuki08320005 585 Tunnel Rd Asheville 288050000 NC US,   --test--MCN-09000630
Prestige Subaru Suzuki 08320005 585 Tunnel Rd Asheville NC 28805 US,   --MDM--MCN-09000630
WOLBERG ELECTRICAL SUPPLY COMPANY I 35 INDUSTRIAL PARK ROAD ALBANY 12206 NY US,   --test--MCN-30243660
Wolberg Electrical Supply Company I 35 Industrial Park Road Albany NY 12206 US,   --MDM--MCN-30243660
ST FRANCIS COUNTY CIRCUIT CLERK 313 S IZARD ST FORREST CITY 72335 AR US,   --test--MCN-81965570
St Francis County Circuit Clerk 313 S Izard St Forrest City AR 72335 US, 

CAROLINA ORTHOPAEDIC & SPORTS 2345 COURT DRIVE GASTONIA 28054 NC US,   --test--MCN-24247020
Carolina Orthopaedic & Sports 2345 Court Drive Gastonia NC 28054 US,   --MDM--MCN-24247020
STAR CONCRETE PUMPING 24223 HUFSMITH KOHRVILLE ROAD TOMBALL 77375 TX US,   --test--MCN-71550775
Star Concrete Pumping 24223 Hufsmith Kohrville Road Tomball TX 77375 US,   --MDM--MCN-71550775
MECKLENBURG COUNTY PUBLIC SCHOOLS 105 MAYFIELD DR BOYDTON 23917 VA US,   --test--MCN-80115450
WHITESIDE MANAGEMENT 2 HENRY ADAMS ST SUITE M-3 SAN FRANCISCO 941030000 CA US,   --test--MCN-09256637
Whiteside Management 2 Henry Adams St Suite M-3 San Francisco CA 94103 US,   --MDM--MCN-09256637
Next Step HealthCare, LLC 392 MAIN ST SACO 04072-1521 ME US,   --test--MCN-59249559
BRANDYWINE REALTY TRUST 555 E LANCASTER AVE WAYNE 19087 PA US,   --test--MCN-83098269
Brandywine Realty & Trust 555 E Lancaster Ave Radnor PA 19087 US,   --MDM--MCN-83098269
Concentric Advisors-hq 4030 Lake Washington Blvd Ne Ste 20 Kirkland 9803300

DAVISON COUNTY 200 E 4TH AVE MITCHELL 573012631 SD US,   --test--MCN-06842294
Davison County 200 E. 4th Ave Mitchell SD 57301 US,   --MDM--MCN-06842294
SUNPOWER INC 430 INDIO WAY SUNNYVALE 94085 CA US,   --test--MCN-04055926
STAFFORD DEVELOPEMENT INC 1805 US HIGHWAY 82 W TIFTON 317938165 GA US,   --test--MCN-53026954
SPEEDWAY MOTORS INC 340 VICTORY LN LINCOLN 68528 NE US,   --test--MCN-69524634
GLOBAL VALUE COMMERCE INC 7320 ACC BLVD RALEIGH 27617 NC US,   --test--MCN-40479299
NEWSOM EYE & LASER CENTER INC 13904 N DALE MABRY HWY STE 200 TAMPA 33618 FL US,   --test--MCN-38802425
Newsom Eye & Laser Center Inc 13904 N Dale Mabry Hwy Ste 200Accounts Payable / Cdw Rwx7180 /cpo Tampa FL 33618 US,   --MDM--MCN-38802425
F.J O'HARA & SONS, INC 7 FID KENNEDY AVE BOSTON 22102321 MA US,   --test--MCN-89641398
F.j. O Hara & Sons 7 Fid Kennedy Ave. Boston MA 2210 US,   --MDM--MCN-89641398
CRAFTWORKS 301 W MAIN ST CHATTANOOGA 37408 TN US,   --test--MCN-27714047
PRAVIS LLC 113 SAVANNAH LOOP MOUNTAIN V

In [36]:
def find_score(chlist, not_found, found_flag,score,i):
    for mcn,value in test_dict.items():
        for j in range(0,2):
            if value == chlist[j][0]:
                if i == mcn:
                    score+=1
#                     gc.collect()
#                     print('Done for test sample', i)
#                     t = time.time()-t1
#                     print("SELFTIMED:", t)
                    
#                     print("{},   --MDM--{}".format(chlist[j][0], mcn))
#                     print("===================================================")
                    found_flag=1
                    break
    if found_flag == 0:
        not_found.append(i)
    
    return score

In [37]:
def fuzzy_extract(test_sample, choices,score, found_flag, not_found,i):
    
    chlist = process.extract(test_sample, choices, limit=2)
    score = find_score(chlist, not_found, found_flag,score,i)
#     print(score)
    return score

In [38]:
import time
import gc
from gevent.pool import Pool
from gevent import monkey
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, wait, as_completed
 
t1 = time.time()
NUM_WORKERS = 10


choices = test_dict.values()

score= 0
found_flag=0
not_found=[]

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = [executor.submit(fuzzy_extract, sample, choices, score, found_flag, not_found, i) for i, sample in golden_dict.items()]
    concurrent.futures.wait(futures)
    
t = time.time()-t1
print("SELFTIMED:", t)
# print("Total matches found are {} out of {}".format(score, len(test_data)))        
# print('accuracy is', 100* score/len(test_data), '%')
# # print('mismatched sample indexes in test data are:')
# # print(not_found)
# print("===================================================")
# print('We are done with 1st level of matching')

SELFTIMED: 25.037545680999756


In [39]:
score=0
for x in as_completed(futures):
    score+= x.result()
#     print(x.result())
print(sum)
print('accuracy is', 100* score/len(test_data), '%')

<built-in function sum>
accuracy is 48.717948717948715 %


In [180]:
not_matched = []
for i in not_found:
    not_matched.append(train_arr[i])

In [181]:
not_matched

['Barwick Cpa 218 Fulford Ave Bel Air 210140000 MD US',
 'Professional Physical Therapy 333 Earle Ovington Blvd  Suite 225 Uniondale 115530000 NY US',
 'Harwood International Inc 3130 N. Harwood Street Dallas 752010000 TX US',
 'Steve Quiroz 10625 N. Lamar Austin 787530000 TX US',
 'ARGOS SCHOOL CORPORATION 410 N FIRST ST ARGOS 46501 IN US',
 'ARGOS SCHOOL CORPORATION 410 N FIRST ST ARGOS 46501 IN US',
 'CITY OF ST. LOUIS, MO - ST. LOUIS ZOO SAINT LOUIS ZOO SAINT LOUIS 63110 MO US',
 'SNIDER INSURANCE GROUP, LLC 1438 WILKESBORO HWY STATESVILLE 28625 NC US',
 'HANSON WALTER & ASSOCIATES, IN 8 BROADWAY SUITE 104 KISSIMMEE 34741 FL US',
 'Mars International 60 Kingsbridge Road Piscataway 88543919 NJ US',
 'SALATA CORPORATION 16720 PARK ROW HOUSTON 77084 TX US',
 'Family Office RPP LLC 450 PARK AVENUE NEW YORK 10022 NY US',
 'LEAKE & WATTS SERVICES, INC 463 HAWTHORNE AVE YONKERS 10705 NY US',
 'BATTERIESPLUS LLC 1325 WALNUT RIDGE DR HARTLAND 53029 WI US',
 'STARPOINTE VENTURES 5030 CAMPUS 

In [182]:
import copy

In [183]:
not_matched_test  = copy.copy(not_matched)

## Super fast string


In [184]:
test_df  = pd.DataFrame({'mcn':mcn_arr_test, 'name-address':train_arr})
test_df.head()

,mcn,name-address
0,MCN-47996044,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...
1,MCN-05952813,Little Apple Toyota Honda 2828 Amherst Ave Man...
2,MCN-67486028,Adp Network Solutions 76002379 16177 Sw 155 Av...
3,MCN-09000630,Prestige Subaru Suzuki08320005 585 Tunnel Rd A...
4,MCN-30243660,WOLBERG ELECTRICAL SUPPLY COMPANY I 35 INDUSTR...


In [185]:
golden_df = pd.DataFrame({'mcn':mcn_arr_golden, 'name-address':golden_flat_list})
golden_df.head()

,mcn,name-address
0,[MCN-47996044],Greg Pollom 218 Fulford Ave Bel Air MD 21014 US
1,[MCN-05952813],Little Apple Toyota Honda 2828 Amherst Ave Man...
2,[MCN-67486028],Adp Network Solutions 76002379 16177 Sw 155 Av...
3,[MCN-09000630],Prestige Subaru Suzuki 08320005 585 Tunnel Rd ...
4,[MCN-30243660],Wolberg Electrical Supply Company I 35 Industr...


In [186]:
names = pd.concat([test_df, golden_df])
len(names)

1150

In [187]:
# names.head(150)

In [188]:
def get_matches_df(sparse_matrix, name_vector, top=1000):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})
        

In [189]:
len(not_matched)

40

In [190]:
for i in range(0, len(golden_flat_list)):
    not_matched.append(golden_flat_list[i])

In [191]:
# not_matched

In [192]:
import re

def ngrams_partial(string, n=1):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in "McDonalds":')
ngrams_partial('McDonalds')


All 3-grams in "McDonalds":


['M', 'c', 'D', 'o', 'n', 'a', 'l', 'd', 's']

In [193]:
from sklearn.feature_extraction.text import TfidfVectorizer

company_names = not_matched
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams_partial)
tf_idf_matrix_partial = vectorizer.fit_transform(company_names)
# company_names

In [194]:
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))


In [425]:
import time
t1 = time.time()
matches_partial = awesome_cossim_top(tf_idf_matrix_partial, tf_idf_matrix_partial.transpose(), 40, 0.50)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.2070331573486328


In [426]:
matches_partial_df = get_matches_df(matches_partial, company_names,len(company_names))
#matches_df.describe
matches_partial_df = matches_partial_df[matches_partial_df['similairity'] <= 0.99999] # Remove all exact matches
# matches_df.sample(20)

# score_partial=0
# for i in range(0,len(not_matched_test)):
#     for j in range(0, len(matches_partial_df)):
#         if (not_matched_test[i] == matches_partial_df['right_side'].tolist()[j]) and (not_matched_test[i] == matches_partial_df['right_side'].tolist()[j]):
#             score_partial+=1
#             break
        
# print('score for mismatched data is', score_partial/len(not_matched_test))


In [427]:
matches_partial_df


,left_side,right_side,similairity
1,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,Greg Pollom 218 Fulford Ave Bel Air MD 21014 US,0.835105
2,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,Alliance Benefit Group Of Michigan 30100 Teleg...,0.790822
3,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,Harrison Group 106 N Baltimore Ave Ocean City ...,0.788953
4,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,Cary Kane Llp 1350 Broadway Rm 1400Suite 1400 ...,0.781168
5,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,Billy Casper Golf Llc 8300 Boone BlvdSuite 350...,0.778093
6,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,"Ibotta 801 California St, Suite 400 Denver CO ...",0.777955
7,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,Air802 2570 Beverly Drive Aurora IL 60502 US,0.777415
8,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,Saban Capital Group 10100 Santa Monica Blvd.Su...,0.776992
9,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,RPM Mortgage 2175 N. California BlvdSuite 1000...,0.776670
10,Barwick Cpa 218 Fulford Ave Bel Air 210140000 ...,Icon Architectural Group 4050 Gardenview DrSte...,0.776062


In [428]:
# not_matched_test[2] == matches_partial_df['right_side'].tolist()[3]

In [429]:
golden_partial_mcn_list = []
for i in range(0, len(matches_partial_df)):
    
    if matches_partial_df['right_side'].tolist()[i] in golden_dict.values():
        
        for mcn, value in golden_dict.items():
    
            if (value == matches_partial_df['right_side'].tolist()[i]):
            
                golden_partial_mcn_list.append(mcn)
                
                break
 
    else:
        golden_partial_mcn_list.append(0)
            
  

In [430]:
golden_partial_mcn_list

['MCN-47996044',
 'MCN-33054732',
 'MCN-77832503',
 'MCN-19360519',
 'MCN-87871304',
 'MCN-79112182',
 'MCN-77584313',
 'MCN-33562561',
 'MCN-52886677',
 'MCN-07027468',
 'MCN-39204903',
 'MCN-17831279',
 'MCN-28259845',
 'MCN-99338228',
 'MCN-09948183',
 'MCN-73858987',
 'MCN-85133510',
 'MCN-32460627',
 'MCN-22935836',
 'MCN-46433696',
 'MCN-87974872',
 'MCN-66459421',
 'MCN-42850293',
 'MCN-18696235',
 'MCN-30333879',
 'MCN-00801612',
 'MCN-36394624',
 'MCN-92441326',
 'MCN-79099330',
 'MCN-04700861',
 'MCN-54188037',
 'MCN-08221586',
 'MCN-00480799',
 'MCN-13307268',
 'MCN-22468291',
 'MCN-80773254',
 'MCN-60783235',
 'MCN-60783235',
 'MCN-87747166',
 'MCN-52283206',
 'MCN-58837910',
 'MCN-41060805',
 'MCN-18696235',
 'MCN-58429627',
 'MCN-08221586',
 'MCN-59558444',
 'MCN-93316098',
 'MCN-74418366',
 'MCN-62093676',
 'MCN-58183385',
 'MCN-65006922',
 'MCN-50643203',
 'MCN-45539173',
 'MCN-34984396',
 'MCN-12607114',
 'MCN-83916057',
 'MCN-30055226',
 'MCN-16993425',
 'MCN-33054732

In [431]:
test_partial_mcn_list = []
for i in range(0, len(matches_partial_df)):
    
    if matches_partial_df['left_side'].tolist()[i] in test_dict.values():
        
        for mcn, value in test_dict.items():
    
            if (value == matches_partial_df['left_side'].tolist()[i]):
            
                test_partial_mcn_list.append(mcn)
                
                break
            
    else:
        test_partial_mcn_list.append(0)
            
  

In [432]:
test_partial_mcn_list

['MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-47996044',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922',
 'MCN-08192922

In [433]:
len(test_partial_mcn_list)

1038

In [434]:
len(golden_partial_mcn_list)

1038

In [435]:
score = 0
score_list=[]
matched_list_step2=[]
for i in range(0, len(test_partial_mcn_list)):
  
    if test_partial_mcn_list[i] == golden_partial_mcn_list[i]:
        if test_partial_mcn_list[i] != 0:
            score+=1
            # score_list is for removing dumplicates from the final matched data
            score_list.extend(([test_partial_mcn_list[i], matches_partial_df['left_side'].tolist()[i]]))
            print("MCN Number in test data is {} and test sample is {}".format(test_partial_mcn_list[i], matches_partial_df['left_side'].tolist()[i]))
            print("MCN Number in golden data is {} and golden sample is {}".format(golden_partial_mcn_list[i], matches_partial_df['right_side'].tolist()[i]))
            print("score is {}".format(matches_partial_df['similairity'].tolist()[i]))
            print('================================================================================') 
            matched_list_step2.append(matches_partial_df['left_side'].tolist()[i])
        if test_partial_mcn_list[i] == test_partial_mcn_list[i+1]:
            
            continue


            
accuracy = 0.5*len(set(score_list))/(len(not_matched_test)) *100 
                  
print('====================================================')   
print('====================================================')  
print("Accuracy on mismatched data is {} %".format(accuracy))


MCN Number in test data is MCN-47996044 and test sample is Barwick Cpa 218 Fulford Ave Bel Air 210140000 MD US
MCN Number in golden data is MCN-47996044 and golden sample is Greg Pollom 218 Fulford Ave Bel Air MD 21014 US
score is 0.8351052468726947
MCN Number in test data is MCN-56591783 and test sample is Harwood International Inc 3130 N. Harwood Street Dallas 752010000 TX US
MCN Number in golden data is MCN-56591783 and golden sample is Harwood International Inc 2501 N Harwood St Ste 1400 Dallas TX 75201 US
score is 0.9635437066930412
MCN Number in test data is MCN-04845501 and test sample is Steve Quiroz 10625 N. Lamar Austin 787530000 TX US
MCN Number in golden data is MCN-04845501 and golden sample is 3rd Coast THIRD COAST AUTOS 10625 N Lamar Blvd Austin TX 78753 US
score is 0.7231098960675955
MCN Number in test data is MCN-00340056 and test sample is CITY OF ST. LOUIS, MO - ST. LOUIS ZOO SAINT LOUIS ZOO SAINT LOUIS 63110 MO US
MCN Number in golden data is MCN-00340056 and golden

In [420]:
# set(score_list)

In [421]:
# not_matched

In [422]:
# not_matched_test

In [423]:
# matches_partial_df.to_csv('test.csv')

In [436]:
matched_list_step2 = list(set(matched_list_step2))
matched_list_step2

['Harwood International Inc 3130 N. Harwood Street Dallas 752010000 TX US',
 'AHSTW CSD 768 S MAPLE ST AVOCA 51521 IA US',
 'Barwick Cpa 218 Fulford Ave Bel Air 210140000 MD US',
 'CITY OF LAKE FOREST 800 N. FIELD DRIVE LAKE FOREST 600450000 IL US',
 'CITY OF ST. LOUIS, MO - ST. LOUIS ZOO SAINT LOUIS ZOO SAINT LOUIS 63110 MO US',
 'Steve Quiroz 10625 N. Lamar Austin 787530000 TX US']

In [118]:
not_matched_test_step2 = [item for item in not_matched_test if item not in matched_list_step2]

In [119]:
len(not_matched_test_step2)

33

In [209]:
# df  = matches_partial_df[matches_partial_df['similairity']>.75]


In [210]:
# [index for index in range(0, len(y_pred)) if y_pred[index]==1]

In [211]:
not_matched_test_step2

['Professional Physical Therapy 333 Earle Ovington Blvd  Suite 225 Uniondale 115530000 NY US',
 'ARGOS SCHOOL CORPORATION 410 N FIRST ST ARGOS 46501 IN US',
 'ARGOS SCHOOL CORPORATION 410 N FIRST ST ARGOS 46501 IN US',
 'SNIDER INSURANCE GROUP, LLC 1438 WILKESBORO HWY STATESVILLE 28625 NC US',
 'HANSON WALTER & ASSOCIATES, IN 8 BROADWAY SUITE 104 KISSIMMEE 34741 FL US',
 'Mars International 60 Kingsbridge Road Piscataway 88543919 NJ US',
 'SALATA CORPORATION 16720 PARK ROW HOUSTON 77084 TX US',
 'Family Office RPP LLC 450 PARK AVENUE NEW YORK 10022 NY US',
 'LEAKE & WATTS SERVICES, INC 463 HAWTHORNE AVE YONKERS 10705 NY US',
 'BATTERIESPLUS LLC 1325 WALNUT RIDGE DR HARTLAND 53029 WI US',
 'STARPOINTE VENTURES 5030 CAMPUS DRIVE SUITE B NEWPORT BEACH 92660 CA US',
 'SALATA CORPORATION 16720 PARK ROW HOUSTON 77084 TX US',
 'Next Step HealthCare, LLC 392 MAIN ST SACO 04072-1521 ME US',
 'Mars International 60 Kingsbridge Road Piscataway 88543919 NJ US',
 'AG PARTNERS IT 101 BROADWAY STREET

In [216]:
test_partial_mcn_list_step2 = []
for i in range(0, len(not_matched_test_step2)):
    for mcn, value in test_dict.items():
           
        if (value == not_matched_test_step2[i]):
                       
                       
            test_partial_mcn_list_step2.append(mcn)
                
            break
            
#     else:
#         test_partial_mcn_list_step2.append(0)
            
  

In [217]:
len(test_partial_mcn_list_step2)

33

In [218]:
test_partial_mcn_list_step2

['MCN-08192922',
 'MCN-84353471',
 'MCN-84353471',
 'MCN-08309418',
 'MCN-04097510',
 'MCN-17657159',
 'MCN-49260309',
 'MCN-60783235',
 'MCN-24327457',
 'MCN-26537322',
 'MCN-17437633',
 'MCN-49260309',
 'MCN-59249559',
 'MCN-17657159',
 'MCN-74098636',
 'MCN-84970424',
 'MCN-99971254',
 'MCN-77879755',
 'MCN-45359866',
 'MCN-48009247',
 'MCN-66029039',
 'MCN-77491597',
 'MCN-70084113',
 'MCN-52985411',
 'MCN-68263648',
 'MCN-69524634',
 'MCN-40479299',
 'MCN-76272798',
 'MCN-50529301',
 'MCN-54537826',
 'MCN-01281248',
 'MCN-28987826',
 'MCN-87123838']

In [252]:
test_data_split_features = []
for i in range(0, len(not_matched_test_step2)):
    z= not_matched_test_step2[i].split(' ')
    z[2:-2] = []
    z = ' '.join(z)
    test_data_split_features.append(z)


In [253]:
test_data_split_features

['Professional Physical NY US',
 'ARGOS SCHOOL IN US',
 'ARGOS SCHOOL IN US',
 'SNIDER INSURANCE NC US',
 'HANSON WALTER FL US',
 'Mars International NJ US',
 'SALATA CORPORATION TX US',
 'Family Office NY US',
 'LEAKE & NY US',
 'BATTERIESPLUS LLC WI US',
 'STARPOINTE VENTURES CA US',
 'SALATA CORPORATION TX US',
 'Next Step ME US',
 'Mars International NJ US',
 'AG PARTNERS MN US',
 'BJ SERVICES TX US',
 'North American CA US',
 'KONRAD BEVERAGE NJ US',
 'CLINIC BY DE US',
 'WATCHDOG BLDG NY US',
 'PIMA MEDICAL AZ US',
 'LUND FOOD MN US',
 'HYDROJET 450 PA US',
 'INSTITUTE FOR NY US',
 'CHARLES JANSEN CA US',
 'SPEEDWAY MOTORS NE US',
 'GLOBAL VALUE NC US',
 'PRAVIS LLC CA US',
 'AMERIPRISE BANK, FL US',
 'VIBE CREDIT MI US',
 'SURRY COUNTY NC US',
 'DS LIQUIDATION TX US',
 'RICHARD C MA US']

In [255]:
choices = golden_dict.values()
score= 0
found_flag=0
not_found={}
for k in 
for i in range(0, len(test_data_split_features)-1):
    found_flag=0
    not_found_iteration = 0
    chlist=process.extract(test_data_split_features[i], choices, limit=25)
    print("{},   --test--{}".format(test_data_split_features[i], test_partial_mcn_list_step2[i]))
    
    for mcn,value in golden_dict.items():
        for j in range(0,24):
            if value == chlist[j][0]:
                if test_partial_mcn_list_step2[i] == mcn:
                    score+=1
                    
                    print("{},   --MDM--{}".format(chlist[j][0], mcn))
                    print("===================================================")
                    found_flag=1
                    break
    if found_flag == 0:
        not_found_iteration.append(i)
        not_found[k] = not_found_iteration
     
print("Total matches found are {} out of {}".format(score, len(test_data_split_features)))        
print('accuracy is', 100* score/len(test_data_split_features), '%')
print('mismatched sample indexes in test data are:')
print(not_found)
print("===================================================")
print('We are done with 1st level of matching')

Professional Physical NY US,   --test--MCN-08192922
Professional Physical Therapy 800 Veteran Memorial HwyCl-100 Hauppauge NY 11788 US,   --MDM--MCN-08192922
ARGOS SCHOOL IN US,   --test--MCN-84353471
Argos Community School District 410 N First St Argos IN 46501 US,   --MDM--MCN-84353471
ARGOS SCHOOL IN US,   --test--MCN-84353471
Argos Community School District 410 N First St Argos IN 46501 US,   --MDM--MCN-84353471
SNIDER INSURANCE NC US,   --test--MCN-08309418
HANSON WALTER FL US,   --test--MCN-04097510
Hanson and Walter 400 W Emmett Street Kissimmee FL 34741 US,   --MDM--MCN-04097510
Mars International NJ US,   --test--MCN-17657159
COUNTY CONCRETE 50 RAILROAD AVE KENVIL NJ 7847 US,   --MDM--MCN-17657159
SALATA CORPORATION TX US,   --test--MCN-49260309
Family Office NY US,   --test--MCN-60783235
LEAKE & NY US,   --test--MCN-24327457
BATTERIESPLUS LLC WI US,   --test--MCN-26537322
STARPOINTE VENTURES CA US,   --test--MCN-17437633
SALATA CORPORATION TX US,   --test--MCN-49260309
Next S

['Barwick Cpa 218 Fulford Ave Bel Air 210140000 MD US',
 'Little Apple Toyota Honda 2828 Amherst Ave Manhattan 665023096 KS US',
 'Adp Network Solutions 76002379 16177 Sw 155 Ave Miami 331870000 FL US',
 'Prestige Subaru Suzuki08320005 585 Tunnel Rd Asheville 288050000 NC US',
 'WOLBERG ELECTRICAL SUPPLY COMPANY I 35 INDUSTRIAL PARK ROAD ALBANY 12206 NY US',
 'ST FRANCIS COUNTY CIRCUIT CLERK 313 S IZARD ST FORREST CITY 72335 AR US',
 'Ramsey Chrysler Jeep76044582 1555 Rte 17 S Ramsey 74460000 NJ US',
 'Presidio Capital Management 12626 High Bluff Dr Ste 145 San Diego 921302074 CA US',
 'THE DAIRY FARM COMPANY 6 MAROON EXT BONITA HOMES CONCEPCION MARIKINA 1800 METRO MANILA PH',
 'Professional Physical Therapy 333 Earle Ovington Blvd  Suite 225 Uniondale 115530000 NY US',
 'BUCCAN 350 S COUNTY RD PALM BEACH 334804481 FL US',
 'LES SCHWAB TIRES 20900 COOLEY RD BEND 97701 OR US',
 'Capitol Cadillac 76071029 6500 Capitol Dr Greenbelt 207700000 MD US',
 'Capitol Cadillac 76071029 6500 Capito

# Number game now

In [262]:
golden_csv.head()

,name,mcn-id,existing-or-new,address1,address2,city,zip,country,temp1,temp2,temp3,temp4,temp5,website,temp6,temp7
0,Greg Pollom,MCN-47996044,EC-Existing,218 Fulford Ave,Bel Air,MD,21014,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Little Apple Toyota Honda,MCN-05952813,R-VAR,2828 Amherst Ave,Manhattan,KS,66502,US,2828 Amherst Ave,Manhattan,KS,66502,US,NaN,NaN,NaN
2,Adp Network Solutions 76002379,MCN-67486028,EC-Existing,16177 Sw 155 Ave,Miami,FL,33187,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Prestige Subaru Suzuki 08320005,MCN-09000630,EC-Existing,585 Tunnel Rd,Asheville,NC,28805,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wolberg Electrical Supply Company I,MCN-30243660,EC-Existing,35 Industrial Park Road,Albany,NY,12206,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [263]:
sample_csv.head()

,Shard ID,eu_mcn,eu_name,eu_address1,eu_address2,eu_address3,eu_address4,eu_city,eu_zip_code,eu_province,eu_state,eu_country
0,n1,MCN-47996044,Barwick Cpa,218 Fulford Ave,NaN,NaN,NaN,Bel Air,210140000,NaN,MD,US
1,n1,MCN-05952813,Little Apple Toyota Honda,2828 Amherst Ave,NaN,NaN,NaN,Manhattan,665023096,NaN,KS,US
2,n1,MCN-67486028,Adp Network Solutions 76002379,16177 Sw 155 Ave,NaN,NaN,NaN,Miami,331870000,NaN,FL,US
3,n1,MCN-09000630,Prestige Subaru Suzuki08320005,585 Tunnel Rd,NaN,NaN,NaN,Asheville,288050000,NaN,NC,US
4,n1,MCN-30243660,WOLBERG ELECTRICAL SUPPLY COMPANY I,35 INDUSTRIAL PARK ROAD,NaN,NaN,NaN,ALBANY,12206,NaN,NY,US


In [342]:
temp = golden_csv['mcn-id'].values
temp = temp.tolist()
match_found_test=[]
for i in range(0, len(sample_csv)):
    if any(sample_csv['eu_mcn'][i] in s for s in temp):
        match_found_test.append(1)
    else:
        match_found_test.append(0)

In [343]:
sample_csv.head()

,Shard ID,eu_mcn,eu_name,eu_address1,eu_address2,eu_address3,eu_address4,eu_city,eu_zip_code,eu_province,eu_state,eu_country,match_found
0,n1,MCN-47996044,Barwick Cpa,218 Fulford Ave,NaN,NaN,NaN,Bel Air,210140000,NaN,MD,US,1
1,n1,MCN-05952813,Little Apple Toyota Honda,2828 Amherst Ave,NaN,NaN,NaN,Manhattan,665023096,NaN,KS,US,1
2,n1,MCN-67486028,Adp Network Solutions 76002379,16177 Sw 155 Ave,NaN,NaN,NaN,Miami,331870000,NaN,FL,US,1
3,n1,MCN-09000630,Prestige Subaru Suzuki08320005,585 Tunnel Rd,NaN,NaN,NaN,Asheville,288050000,NaN,NC,US,1
4,n1,MCN-30243660,WOLBERG ELECTRICAL SUPPLY COMPANY I,35 INDUSTRIAL PARK ROAD,NaN,NaN,NaN,ALBANY,12206,NaN,NY,US,1


In [344]:
match_found_test

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

In [346]:
sample_csv['match_found'] = match_found_test

In [345]:
# sample_csv.head()

In [449]:
choices = golden_dict.values()
accuracy = []
not_found= {}
for k in range(25,40):
    not_found_iteration = []
    found_flag=0
    score= 0
    
    for i in range(0, len(test_data)-1):
        found_flag=0
        chlist=process.extract(test_data[i], choices, limit=k)
#         print("{},   --test--{}".format(test_data[i], mcn_arr_test[i]))

        for mcn,value in golden_dict.items():
            for j in range(0,k):
                if value == chlist[j][0]:
                    if mcn_arr_test[i] == mcn:
                        score+=1

#                         print("{},   --MDM--{}".format(chlist[j][0], mcn))
#                         print("===================================================")
                        found_flag=1
                        break
        if found_flag == 0:
            not_found_iteration.append(i)
            not_found[k] = not_found_iteration
    accuracy.append(100* score/len(test_data))
#     print("Total matches found are {} out of {}".format(score, len(test_data)))        
    print('accuracy is', 100* score/len(test_data), '%')
    print('mismatched sample indexes in test data are:')
#     print(not_found)
    
    print("===================================================")
    print('We are done with 1st level of matching')
print(accuracy)
print(not_found)

accuracy is 65.8119658119658 %
mismatched sample indexes in test data are:
We are done with 1st level of matching
accuracy is 67.52136752136752 %
mismatched sample indexes in test data are:
We are done with 1st level of matching
accuracy is 67.52136752136752 %
mismatched sample indexes in test data are:
We are done with 1st level of matching
accuracy is 67.52136752136752 %
mismatched sample indexes in test data are:
We are done with 1st level of matching
accuracy is 67.52136752136752 %
mismatched sample indexes in test data are:
We are done with 1st level of matching
accuracy is 69.23076923076923 %
mismatched sample indexes in test data are:
We are done with 1st level of matching
accuracy is 69.23076923076923 %
mismatched sample indexes in test data are:
We are done with 1st level of matching
accuracy is 70.94017094017094 %
mismatched sample indexes in test data are:
We are done with 1st level of matching
accuracy is 70.94017094017094 %
mismatched sample indexes in test data are:
We ar

In [447]:
chlist=process.extract(test_data[1], choices, limit=5)

In [448]:
chlist

[('Little Apple Toyota Honda 2828 Amherst Ave Manhattan KS 66502 US', 92),
 ('Buccan 350 S County Rd Palm Beach FL 33480 US', 86),
 ('Holiday Inn 2965 E Main St Alice TX 78332 US', 86),
 ('AZITC 5751 e 13th st tucson AZ 85711 US', 86),
 ('Newsom Eye & Laser Center Inc 13904 N Dale Mabry Hwy Ste 200Accounts Payable / Cdw Rwx7180 /cpo Tampa FL 33618 US',
  86)]

In [348]:
# test_data[1]

In [393]:
TrueP = {}
FalseP = {}
TP = 0
FP =0
for key, value in not_found.items():
    for j in value:
        if match_found_test[j] == 1:
            TP+=1
        else:
            FP+=1
            
    TrueP[key] = TP
    FalseP[key] = FP
    
    TP = 0
    FP =0

In [394]:
TrueP

{0: 102,
 1: 51,
 2: 50,
 3: 49,
 4: 49,
 5: 48,
 6: 48,
 7: 46,
 8: 46,
 9: 46,
 10: 46,
 11: 46,
 12: 46,
 13: 46,
 14: 45,
 15: 45,
 16: 43,
 17: 43,
 18: 43,
 19: 39,
 20: 38,
 21: 38,
 22: 37,
 23: 35,
 24: 33}

In [395]:
FalseP

{0: 14,
 1: 8,
 2: 8,
 3: 8,
 4: 8,
 5: 8,
 6: 8,
 7: 8,
 8: 8,
 9: 8,
 10: 8,
 11: 8,
 12: 8,
 13: 8,
 14: 7,
 15: 7,
 16: 7,
 17: 7,
 18: 7,
 19: 7,
 20: 7,
 21: 7,
 22: 7,
 23: 7,
 24: 7}

In [471]:
# for i in range(0, len(not_found)):
#     print(len(not_found[i]))